# Project 2

## Summary of Project

# Hypothesis or A/B Testing

## Three Tests
1. Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?
2. ?
3. ?

In [1]:
#import libraries and functions

In [2]:
#get sql database

In [3]:
#function to pull sql queries and return a pandas table

In [ ]:
#query 1

In [4]:
#statistical analysis 1

In [ ]:
#graphs 1

In [ ]:
#query 2

In [6]:
#statistical analysis 2

In [ ]:
#graphs 2

In [7]:
#query 3

In [ ]:
#statistical analysis 3

In [5]:
#graphs 3

# Conclusion